In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
#Read CSV file

city_data = pd.read_csv("Resources/weather_df.csv", encoding= "utf-8")
city_data.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,punta arenas,40,CL,1600112454,61,-53.15,-70.92,46.4,27.51
1,1,port elizabeth,0,ZA,1600112400,76,-33.92,25.57,55.4,24.16
2,2,tuktoyaktuk,90,CA,1600112553,93,69.45,-133.04,35.6,10.29
3,3,nikolskoye,11,RU,1600112554,87,59.70,30.79,51.8,8.95
4,4,ushuaia,20,AR,1600112486,64,-54.80,-68.30,42.8,16.11


In [3]:
#Add values for HeatMap
locations= city_data[["Lat", "Lng"]]
humidity_weight = city_data["Humidity"]
humidity_max = humidity_weight.max()

In [4]:
#Add HeatMap
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations, weights= humidity_weight, dissipating=False, max_intensity=humidity_max, point_radius= 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#Creating clean DF with specific requirements
clean_df = city_data.loc[(city_data["Max Temp"]>70) & (city_data["Max Temp"]<80) & (city_data["Cloudiness"]==0) &(city_data["Wind Speed"]<10), :]
clean_df=clean_df.dropna()
clean_df = clean_df.drop(["Unnamed: 0"], axis =1)
clean_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
8,bardiyah,0,LY,1600112561,77,31.76,25.09,76.53,7.81
68,tatawin,0,TN,1600112613,57,32.93,10.45,77.00,6.93
91,madimba,0,TZ,1600112661,90,-10.40,40.33,72.97,5.50
177,darnah,0,LY,1600112769,78,32.77,22.64,75.54,5.32
203,wana,0,PK,1600112800,32,32.30,69.57,77.79,3.80


In [6]:
#Hotel Mapping
hotel_df = clean_df
hotel_df.reset_index(drop= True, inplace=True)
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,bardiyah,0,LY,1600112561,77,31.76,25.09,76.53,7.81
1,tatawin,0,TN,1600112613,57,32.93,10.45,77.00,6.93
2,madimba,0,TZ,1600112661,90,-10.40,40.33,72.97,5.50
3,darnah,0,LY,1600112769,78,32.77,22.64,75.54,5.32
4,wana,0,PK,1600112800,32,32.30,69.57,77.79,3.80
5,miandrivazo,0,MG,1600112826,62,-19.52,45.47,77.18,1.25
6,asfi,0,MA,1600112830,86,32.30,-9.24,70.07,9.80
7,zhob,0,PK,1600112835,34,31.34,69.45,78.82,9.08
8,qabaqcol,0,AZ,1600112973,73,41.72,46.26,71.60,9.17
9,pierrelatte,0,FR,1600112974,60,44.38,4.70,75.99,3.40


In [7]:
#Create a list for the hotels
hotels=[]

params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key,
}

#get the location for citys
for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']
#add the location to the parameters
    params["location"]=f'{lat},{lng}'
#set up url and API to request data
    url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    print(f"Results for {index}:{row['City']}.")
    response = requests.get(url, params=params).json()
#add results to a variable
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"]= results[0]['name']
    except:
        print("Missing information...skip") 
  

Results for 0:bardiyah.
Closest hotel is Hotel Burdy.
Results for 1:tatawin.
Closest hotel is Sangho Privilege Tataouine.
Results for 2:madimba.
Missing information...skip
Results for 3:darnah.
Closest hotel is فندق النعيم.
Results for 4:wana.
Missing information...skip
Results for 5:miandrivazo.
Closest hotel is Princesse Tsiribihina.
Results for 6:asfi.
Closest hotel is Hôtel Farah Safi.
Results for 7:zhob.
Closest hotel is Bolan Hotel.
Results for 8:qabaqcol.
Missing information...skip
Results for 9:pierrelatte.
Closest hotel is Mansion De Digoine.
Results for 10:mantua.
Closest hotel is B&B Palazzo Castiglioni.
Results for 11:omboue.
Closest hotel is Hôtel Georges Faustin Aurat.
Results for 12:bud.
Missing information...skip
Results for 13:mahon.
Closest hotel is Hotel Rural Biniarroca.
Results for 14:manbij.
Closest hotel is Manbej City Hotel.
Results for 15:nacala.
Closest hotel is INDICO EXECUTIVO HOTEL.


In [8]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,bardiyah,0,LY,1600112561,77,31.76,25.09,76.53,7.81,Hotel Burdy
1,tatawin,0,TN,1600112613,57,32.93,10.45,77.00,6.93,Sangho Privilege Tataouine
2,madimba,0,TZ,1600112661,90,-10.40,40.33,72.97,5.50,NaN
3,darnah,0,LY,1600112769,78,32.77,22.64,75.54,5.32,فندق النعيم
4,wana,0,PK,1600112800,32,32.30,69.57,77.79,3.80,NaN
5,miandrivazo,0,MG,1600112826,62,-19.52,45.47,77.18,1.25,Princesse Tsiribihina
6,asfi,0,MA,1600112830,86,32.30,-9.24,70.07,9.80,Hôtel Farah Safi
7,zhob,0,PK,1600112835,34,31.34,69.45,78.82,9.08,Bolan Hotel
8,qabaqcol,0,AZ,1600112973,73,41.72,46.26,71.60,9.17,NaN
9,pierrelatte,0,FR,1600112974,60,44.38,4.70,75.99,3.40,Mansion De Digoine


In [9]:

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))